In [233]:
import pandas as pd
import json
import numpy as np
from itertools import permutations
from scipy.spatial.distance import cdist
import pandas as pd
import numpy as np
from itertools import permutations
from math import sqrt
import random

In [234]:
# Parámetros globales

with open('../scheduling/MC_Supply_Chain_Solution.json', 'r') as f:
    data = json.load(f)
    demand_df = pd.DataFrame(data['best_solution'], columns=['day', 'polygon', 'specie', 'supplier', 'amount'])

polygon_df = pd.read_csv('../setup/ha.csv')


In [235]:
# Helper Functions
def euclidean_distance(p1, p2):
    return sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def compute_time_matrix(polygon_df):
    polygons = polygon_df['Poligono'].tolist()
    coords = polygon_df.set_index('Poligono')[['X', 'Y']].to_dict('index')
    time_data = []
    for i in polygons:
        for j in polygons:
            dist = euclidean_distance(list(coords[i].values()), list(coords[j].values())) / 1000 # KM
            time_hours = dist / 10
            time_minutes = np.ceil(time_hours * 60)
            time_data.append({
                'origin' : i,
                'target' : j,
                'time' : time_minutes
            })
    time_df = pd.DataFrame(time_data)
    # time_df = time_df[time_df['time'] != 0]
    time_df = time_df.pivot(index='origin', columns='target', values='time')

    return time_df

In [236]:
demand_day = demand_df[demand_df['day'] == 1]
demand_day = demand_day.groupby('polygon').agg({'amount' : 'sum'}).reset_index()

In [237]:
compute_time_matrix(polygon_df)

target,1,3,4,5,16,17,18,19,20,23,24,25,26
origin,,,,,,,,,,,,,
1,0.0,3.0,2.0,2.0,8.0,8.0,8.0,9.0,10.0,9.0,9.0,11.0,12.0
3,3.0,0.0,2.0,3.0,8.0,7.0,7.0,9.0,8.0,8.0,8.0,11.0,13.0
4,2.0,2.0,0.0,2.0,7.0,7.0,7.0,8.0,9.0,8.0,8.0,10.0,12.0
5,2.0,3.0,2.0,0.0,6.0,7.0,7.0,8.0,9.0,8.0,8.0,9.0,11.0
16,8.0,8.0,7.0,6.0,0.0,2.0,3.0,2.0,9.0,6.0,5.0,4.0,5.0
17,8.0,7.0,7.0,7.0,2.0,0.0,2.0,3.0,7.0,4.0,3.0,4.0,6.0
18,8.0,7.0,7.0,7.0,3.0,2.0,0.0,4.0,6.0,3.0,2.0,5.0,7.0
19,9.0,9.0,8.0,8.0,2.0,3.0,4.0,0.0,10.0,7.0,5.0,2.0,4.0
20,10.0,8.0,9.0,9.0,9.0,7.0,6.0,10.0,0.0,4.0,5.0,11.0,12.0


In [238]:
HQ_POLYGON = 18
MAX_TIME = 360  # in minutes
LOADING_TIME = 30
UNLOADING_TIME = 30
TRUCK_CAPACITY = 1000  # example capacity, adjust as needed

class VRP:
    def __init__(self, demand_df, polygon_df):
        self.demand_df = demand_df.copy()
        self.polygon_df = polygon_df
        self.time_df = compute_time_matrix(polygon_df)

        self.current_state = {
            'location': HQ_POLYGON,
            'time': 0,
            'load': 0,
            'inventory': [],
            'delivered': []
        }
        self.action_history = []
        self.remaining = self.demand_df.copy()
        self.goal_state = None  # Unused in this context

    def get_state(self):
        return self.current_state.copy()

    def get_actions(self, state):
        actions = []

        # Option 1: Load from warehouse (only at HQ)
        if state['location'] == HQ_POLYGON and state['time'] + LOADING_TIME <= MAX_TIME:
            actions.append({'type': 'load'})

        # Option 2: Deliver (if inventory not empty)
        for i, row in enumerate(state['inventory']):
            polygon = row['polygon']
            if state['location'] != polygon:
                travel_time = self.time_df.at[state['location'], polygon]
                total_time = travel_time + UNLOADING_TIME + self.time_df.loc[polygon, HQ_POLYGON]
                if state['time'] + total_time <= MAX_TIME:
                    actions.append({'type': 'deliver', 'order': row})
            else:
                if state['time'] + UNLOADING_TIME + self.time_df.loc[state['location'], HQ_POLYGON] <= MAX_TIME:
                    actions.append({'type': 'deliver', 'order': row})

        # Option 3: Return to HQ
        if state['location'] != HQ_POLYGON:
            travel_time = self.time_df.loc[state['location'], HQ_POLYGON]
            if state['time'] + travel_time <= MAX_TIME:
                actions.append({'type': 'return'})

        return actions

    def protocol(self, actions, state):
        # Very basic strategy: deliver nearest possible order or return/load
        for action in actions:
            if action['type'] == 'deliver':
                return action
        for action in actions:
            if action['type'] == 'load':
                return action
        for action in actions:
            if action['type'] == 'return':
                return action
        return None

    def apply_action(self, action):
        state = self.current_state
        if action['type'] == 'load':
            # Greedy fill of truck from warehouse
            capacity = TRUCK_CAPACITY
            new_inventory = []
            new_remaining = []
            for _, row in self.remaining.iterrows():
                amount = row['amount']
                if amount <= capacity:
                    new_inventory.append(row.to_dict())
                    capacity -= amount
                else:
                    new_remaining.append(row.to_dict())

            self.current_state['inventory'] = new_inventory
            self.remaining = pd.DataFrame(new_remaining)
            self.current_state['load'] = TRUCK_CAPACITY - capacity
            self.current_state['time'] += LOADING_TIME
            self.action_history.append(('load', len(new_inventory)))

        elif action['type'] == 'deliver':
            order = action['order']
            travel_time = self.time_df.loc[state['location'], order['polygon']]
            self.current_state['time'] += travel_time + UNLOADING_TIME
            self.current_state['location'] = order['polygon']
            self.current_state['inventory'].remove(order)
            self.current_state['load'] -= order['amount']
            self.current_state['delivered'].append(order)
            self.action_history.append(('deliver', order))

        elif action['type'] == 'return':
            travel_time = self.time_df.loc[state['location'], HQ_POLYGON]
            self.current_state['time'] += travel_time
            self.current_state['location'] = HQ_POLYGON
            self.action_history.append(('return',))

    def run(self):
        while self.current_state['time'] < MAX_TIME:
            actions = self.get_actions(self.current_state)
            if not actions:
                break
            selected_action = self.protocol(actions, self.current_state)
            if not selected_action:
                break
            self.apply_action(selected_action)

        return self.current_state, self.action_history


In [239]:
optimizer = VRP(demand_df, polygon_df)
optimizer.run()

({'location': 18,
  'time': 334.0,
  'load': 327,
  'inventory': [{'day': 1,
    'polygon': 24,
    'specie': 'Opuntia robusta',
    'supplier': 'Moctezuma',
    'amount': 327}],
  'delivered': [{'day': 1,
    'polygon': 1,
    'specie': 'Agave lechuguilla',
    'supplier': 'Laguna seca',
    'amount': 178},
   {'day': 1,
    'polygon': 20,
    'specie': 'Agave salmiana',
    'supplier': 'Laguna seca',
    'amount': 216},
   {'day': 1,
    'polygon': 19,
    'specie': 'Agave striata',
    'supplier': 'Venado',
    'amount': 162},
   {'day': 1,
    'polygon': 3,
    'specie': 'Agave lechuguilla',
    'supplier': 'Laguna seca',
    'amount': 264},
   {'day': 1,
    'polygon': 1,
    'specie': 'Agave scabra',
    'supplier': 'Moctezuma',
    'amount': 178},
   {'day': 1,
    'polygon': 5,
    'specie': 'Opuntia streptacanta',
    'supplier': 'Venado',
    'amount': 385},
   {'day': 1,
    'polygon': 4,
    'specie': 'Agave striata',
    'supplier': 'Venado',
    'amount': 264}]},
 [('load